In [47]:
import glob

import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import *

# import SyllableTokenizer() method from nltk
from nltk import word_tokenize

import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import svm
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score


from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory
factorys = StemmerFactory()
stemmer = factorys.create_stemmer()


In [11]:
paths = glob.glob('../../bt.1.0/docs/*')

In [12]:
len(paths)

594

In [20]:
def data_preparation(paths):
    files = []
    for path in paths:
        f = open(path, "r")
        print(f)
        files.append([f.read()])

    labels = []
    for path in paths:
        if path[-3:-1] == 'is':
            labels.append('Israeli')
        elif path[-3:-1] == 'al':
            labels.append('Palestinian')
        else:
            pass
    return files, labels

In [32]:
text, labels = data_preparation(paths)

In [33]:
X = np.array(text)
y = np.array(labels)

In [76]:

stop_words = stopwords.words('english')
stemmer = PorterStemmer()

def clean_text(text, stop_words, stemmer):
    return " ".join([stemmer.stem(word) for word in word_tokenize(text=text[0], language='english', preserve_line=False) 
                    if word not in stop_words and not word.isnumeric()])

In [82]:
X = np.array([clean_text(text, stop_words, stemmer) for text in X])

In [80]:
kfold = KFold(3, shuffle=True, random_state=33)
i = 1
acc_score = []
for train_idx, test_idx in kfold.split(X):
    X_train = X[train_idx]
    y_train = y[train_idx]

    X_test = X[test_idx]
    y_test = y[test_idx]

    vectorizer = TfidfVectorizer()
    X_train = vectorizer.fit_transform(X_train)
    X_test = vectorizer.transform(X_test)


    classifier_rbf = svm.SVC()
    classifier_rbf.fit(X_train, y_train)
    
    pred_values = classifier_rbf.predict(X_test)
     
    acc = accuracy_score(pred_values , y_test)
    acc_score.append(acc)
     
avg_acc_score = sum(acc_score)/k
 
print('accuracy of each fold - {}'.format(acc_score))
print('Avg accuracy : {}'.format(avg_acc_score))

accuracy of each fold - [0.9122807017543859, 0.9473684210526315, 0.9736842105263158, 0.9736842105263158, 0.9557522123893806, 0.9122807017543859, 0.9473684210526315, 0.9736842105263158, 0.9736842105263158, 0.9557522123893806]
Avg accuracy : 0.952553951249806 